## Simple model
---

The problem can be stated like this:

We have $n$ subjects that produce and consume energy we want to distribute excess energy to those subjects who produced less than they need. We have 5 steps to distribute all the excess energy. We need to come up  with allocation key, here we can imagine vector $p$ satisfying $\sum_{i=1}^{n} p_{i} = 1$ and $p_{i}  \geq 0$ $\forall i \in \{1, ..., n\}$. Vector $p$ represents proportion of excess energy for each subject. *The question is how to determine p for whole month (k timestamps) such that leftover energy will be minimized?*

Let $S_t$ be a vector of consumption of all subject in time $t$, $V_t$ be production. Then let's define $E_t = max\{V_t - S_t, \bold{0}\}$ and $D_t = max\{S_t - V_t, \bold{0}\}$. And decision variable $X$, which will give how much energy will be allocated to each subject.

### Fixed Rate approach

Let $Y$ be a vector of total consumed energy in past year. Then proportional vector $p_Y(i) = \frac{Y_i}{\sum_{i=1}^{n} Y_i}$ is fixed rate which we use for energy allocation. $X_t = (\sum_{i=1}^{n} E_t(i)) * p_Y$.

### Waterfall

In a waterfall approach we calculate vector $p_{W} = \frac{D_{t}(i)}{\sum_{i=1}^{n}D_{t}(i)}$. Allocated energy is then computed by $X_t = (\sum_{i=1}^{n} E_t(i)) * p_{W}$.

### Linear programming


***MIN*** $(\sum_{i=1}^{n} E_t(i)) - (\bold{1} * X_t)$, this is equivalent with ***MAX*** $- (\bold{1} * X_t)$

s.t.
- $(\bold{1} * X_t) \leq (\sum_{i=1}^{n} E_t(i))$, We cannot share more energy than we have.
- $\forall i \in \{1, ..., n\}~X_{t}(i) \leq D_{t}(i)$, We don't want to share more than needed.
- $\forall i \in \{1, ..., n\}~0 \leq x_{t}(i)$, negative energy doesn't make sense.

Vector solving this problem is the solution to allocation problem.

In [1]:
import numpy as np
import pandas as pd
import time
from scipy.optimize import linprog

In [2]:
excess  = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/excess.csv')
deficit = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/deficit.csv')
yearly_cons = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/yearly_consumption.csv')

excess['timestamp'] = pd.to_datetime(excess['timestamp'])
excess.set_index('timestamp', inplace=True)
deficit['timestamp'] = pd.to_datetime(deficit['timestamp'])
deficit.set_index('timestamp', inplace=True)

deficit = deficit.astype(float)
excess = excess.astype(float)

In [3]:
excess.describe()

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
count,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.0,35036.0,35036.0,35036.0,35036.000000
mean,0.006574,0.000287,0.007803,0.003103,0.001201,0.003194,0.000719,0.009478,0.000774,0.0,0.0,0.0,0.0,0.007638
std,0.012364,0.000900,0.013330,0.005759,0.002073,0.005759,0.001341,0.019287,0.001924,0.0,0.0,0.0,0.0,0.012950
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
75%,0.006756,0.000000,0.009632,0.003159,0.001639,0.003699,0.000789,0.007602,0.000000,0.0,0.0,0.0,0.0,0.009404
max,0.056772,0.006434,0.057481,0.024997,0.008946,0.025666,0.006022,0.095631,0.011594,0.0,0.0,0.0,0.0,0.056154


In [4]:
deficit.describe()

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
count,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.0
mean,0.001473,0.002241,0.000284,0.000659,0.000146,0.000327,0.000452,0.011466,0.002551,0.004472,0.000725,0.000067,0.032603,0.0
std,0.002225,0.001602,0.000281,0.000672,0.000244,0.000596,0.000688,0.015986,0.001957,0.005904,0.000424,0.000044,0.009613,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000053,0.000000,0.009989,0.0
25%,0.000000,0.001314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000979,0.000178,0.000415,0.000031,0.024657,0.0
50%,0.000785,0.002117,0.000326,0.000620,0.000069,0.000118,0.000018,0.003615,0.002265,0.000772,0.000620,0.000062,0.029568,0.0
75%,0.001226,0.003002,0.000525,0.001078,0.000131,0.000271,0.000839,0.018401,0.004523,0.009089,0.000970,0.000097,0.042373,0.0
max,0.018844,0.010554,0.001226,0.005282,0.002349,0.007901,0.004165,0.066221,0.008953,0.027600,0.002253,0.000258,0.059035,0.0


In [5]:
def sequential_simple_model(excess, deficit, method, steps, yearly_cons):

    result_df = pd.DataFrame(columns=['Total Excess','Excess - Deficit', 'Optimal Residual', 'True Residual', 'Average steps'])
    result_df.index.name = "Timestamp"

    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    for time_stamp in deficit.index:
        e_t = excess.loc[time_stamp]
        d_t = deficit.loc[time_stamp]
        total_allocation, step = method(e_t, d_t, steps, yearly_cons)
        result_df.loc[time_stamp] = [excess.loc[time_stamp].sum(),
                                    excess.loc[time_stamp].sum() - deficit.loc[time_stamp].sum(),
                                    np.max([deficit.loc[time_stamp].sum() - excess.loc[time_stamp].sum(), 0]),
                                    deficit.loc[time_stamp].sum() - total_allocation,
                                    step]
    
    return result_df

def waterfall(e_t, d_t, steps, yearly_cons):
        total_deficit = d_t.sum()
        total_excess = e_t.sum()
        total_allocation = 0

        for step in range(steps):
            if ((total_deficit > 0) and (total_excess > 0)):
            
                p = d_t/total_deficit
                allocation = np.minimum(total_excess * p, d_t)
                d_t = d_t - allocation

                proportion = e_t / np.sum(e_t)
                proportion = np.nan_to_num(proportion, nan=0.0)
                R_t = np.outer(proportion, allocation)
                e_t = e_t - np.sum(R_t, axis=1)

                total_allocation += allocation.sum()
                total_deficit -= allocation.sum()
                total_excess  -= allocation.sum()

            else:
                break
        
        return total_allocation, step

def fixed_rate(e_t, d_t, steps, yearly_cons):
    yearly_cons_dict = yearly_cons.set_index('Column')['Y_cons'].to_dict()
    p = pd.Series(yearly_cons_dict).reindex(deficit.columns, fill_value=0)
    p /= p.sum()

    total_deficit = d_t.sum()
    total_excess = e_t.sum()
    total_allocation = 0

    for step in range(steps):
        if ((total_deficit > 0) and (total_excess > 0)):
            allocation = np.minimum(total_excess * p, d_t)
            d_t = d_t - allocation
            
            proportion = e_t / np.sum(e_t)
            R_t = np.outer(proportion, allocation)
            e_t = e_t - np.sum(R_t, axis=1)

            total_allocation += allocation.sum()
            total_deficit -= allocation.sum()
            total_excess  -= allocation.sum()

        else:
            break
    
    return total_allocation, step


def lin_prog(e_t, d_t, steps, yearly_cons):
    n = len(d_t)

    total_deficit = d_t.sum()
    total_excess = e_t.sum()
    total_allocation = 0

    for step in range(steps):

        if ((total_deficit > 0) and (total_excess > 0)):

            # Objective: minimize -sum(x) i.e. maximize sum(x)
            c = -np.ones(n)

            # Constraint 1: sum(x) <= sum(E_t)
            A_ub = [np.ones(n)]
            b_ub = [np.sum(e_t)]

            # Constraint 2: x_i <= D_t(i) for each subject i
            A_ub.extend(np.eye(n))
            b_ub.extend(d_t)

            A_ub = np.array(A_ub)
            b_ub = np.array(b_ub)
            bounds = [(0, None)] * n

            res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

            if res.success:
                x_opt = res.x  # Optimal energy allocation
            else:
                x_opt = np.zeros(n)

            allocation = np.minimum(x_opt, d_t)
            d_t = d_t - allocation
            
            proportion = e_t / np.sum(e_t)
            R_t = np.outer(proportion, allocation)
            e_t = e_t - np.sum(R_t, axis=1)

            total_allocation += allocation.sum()
            total_deficit -= allocation.sum()
            total_excess  -= allocation.sum()

        else:
            break

    return total_allocation, step

In [6]:

fixed_df = sequential_simple_model(excess, deficit, fixed_rate, 5, yearly_cons)
fixed_df.agg({'Optimal Residual' : 'sum',
              'True Residual': 'sum',
              'Average steps':'mean'})

Optimal Residual    1479.124137
True Residual       1484.279223
Average steps          1.497317
dtype: float64

In [7]:
waterfall_df = sequential_simple_model(excess, deficit, waterfall, 1, yearly_cons)
waterfall_df.agg({'Optimal Residual' : 'sum',
              'True Residual': 'sum',
              'Average steps':'mean'})

Optimal Residual    1479.124137
True Residual       1479.124137
Average steps          0.000000
dtype: float64

In [8]:
lin_prog_df = sequential_simple_model(excess, deficit, lin_prog, 1, yearly_cons)
lin_prog_df.agg({'Optimal Residual' : 'sum',
              'True Residual': 'sum',
              'Average steps':'mean'})

Optimal Residual    1479.124137
True Residual       1479.124137
Average steps          0.000000
dtype: float64